# Data Loading with LangChain: CSV vs Excel

LangChain provides convenient tools for loading and processing tabular data from various sources, including CSV and Excel files. Understanding the differences and use cases for each format helps in selecting the right approach for your workflow.

## CSV Loading

- **Format:** Plain text, comma-separated values.
- **Advantages:**
    - Lightweight and widely supported.
    - Easy to read and write programmatically.
    - Suitable for simple tabular data.
- **Limitations:**
    - No support for multiple sheets.
    - Limited data types (everything is text).
    - No cell formatting or formulas.

**LangChain Example:**
```python
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="data.csv")
documents = loader.load()
```

## Excel Loading

- **Format:** Binary or XML-based spreadsheet (.xls, .xlsx).
- **Advantages:**
    - Supports multiple sheets.
    - Rich data types, cell formatting, and formulas.
    - Useful for complex data and reporting.
- **Limitations:**
    - Larger file size.
    - Requires additional libraries (e.g., `openpyxl`, `xlrd`).
    - Slightly slower to process.

**LangChain Example:**
```python
from langchain.document_loaders.excel import UnstructuredExcelLoader

loader = UnstructuredExcelLoader("data.xlsx")
documents = loader.load()
```

## Comparison Table

| Feature            | CSV Loader                | Excel Loader                |
|--------------------|--------------------------|-----------------------------|
| File Format        | .csv                     | .xls, .xlsx                 |
| Multiple Sheets    | ❌                        | ✅                           |
| Data Types         | Text only                | Rich types (numbers, dates) |
| Formatting         | ❌                        | ✅                           |
| Performance        | Fast                     | Moderate                    |
| Library Support    | Built-in Python, pandas  | openpyxl, xlrd, pandas      |

## When to Use Which?

- **Use CSV Loader:** For simple, flat data tables without formatting or formulas.
- **Use Excel Loader:** For complex spreadsheets with multiple sheets, formatting, or formulas.

LangChain makes it easy to integrate both formats into your data pipelines, allowing flexible document loading for downstream processing and analysis.

In [7]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

csv_loader = CSVLoader(file_path="data/csv_files/hotel_bookings.csv", 
encoding="utf-8",
csv_args={'delimiter': ',',
'quotechar': '"'}
)
csv_documents = csv_loader.load()
print(f"Loaded {len(csv_documents)} documents using CSVLoader.")
print("Sample document content:")
print(csv_documents[0])  # Print the first document's content
# Metadata Example
print("\nSample document metadata:")
for key, value in csv_documents[0].metadata.items():
    print(f"{key}: {value}")
    

Loaded 119390 documents using CSVLoader.
Sample document content:
page_content='hotel: Resort Hotel
is_canceled: 0
lead_time: 342
arrival_date_year: 2015
arrival_date_month: July
arrival_date_week_number: 27
arrival_date_day_of_month: 1
stays_in_weekend_nights: 0
stays_in_week_nights: 0
adults: 2
children: 0
babies: 0
meal: BB
country: PRT
market_segment: Direct
distribution_channel: Direct
is_repeated_guest: 0
previous_cancellations: 0
previous_bookings_not_canceled: 0
reserved_room_type: C
assigned_room_type: C
booking_changes: 3
deposit_type: No Deposit
agent: NULL
company: NULL
days_in_waiting_list: 0
customer_type: Transient
adr: 0
required_car_parking_spaces: 0
total_of_special_requests: 0
reservation_status: Check-Out
reservation_status_date: 2015-07-01' metadata={'source': 'data/csv_files/hotel_bookings.csv', 'row': 0}

Sample document metadata:
source: data/csv_files/hotel_bookings.csv
row: 0


# CSV Processing Strategies in LangChain

LangChain supports multiple strategies for loading and processing CSV data, each suited to different use cases and requirements. Here, we compare two common approaches: **row-based loading using CSVLoader** and **custom document creation using a user-defined loader**.

---

## 1. Row-Based Loading (CSVLoader)

- **Approach:**  
    Each row in the CSV file is loaded as a separate `Document` object.
- **Implementation:**  
    Utilizes the built-in `CSVLoader` from LangChain, which automatically parses each row and attaches metadata (such as source file and row index).
- **Advantages:**  
    - Simple and fast for standard tabular data.
    - Metadata includes row number and source file for easy traceability.
    - Minimal code required.
    - Good for record lookups
- **Limitations:**  
    - Limited customization of document content and metadata.
    - May not handle complex formatting or nested data well.
    - Loses table context

**Example:**
```python
from langchain_community.document_loaders import CSVLoader

csv_loader = CSVLoader(file_path="data/csv_files/Products.csv")
csv_documents = csv_loader.load()
```
Each document contains the content of a single row and basic metadata.

---

## 2. Custom Loader

- **Approach:**  
    Define a custom function to process each row, allowing for tailored document content and metadata.
- **Implementation:**  
    Read the CSV with pandas, iterate over rows, and create `Document` objects with custom formatting and metadata fields.
- **Advantages:**  
    - Full control over document structure and metadata.
    - Can combine fields, add computed values, or filter data as needed.
    - Supports advanced use cases (e.g., category tagging, custom summaries).
    - Creates Summaries
    - Rich Metadata & better for Q&A
- **Limitations:**  
    - Requires more code and maintenance.
    - Slightly slower due to manual processing.

**Example:**
```python
def custom_csv_loader(filepath: str) -> List[Document]:
        df = pd.read_csv(filepath)
        documents = []
        for idx, row in df.iterrows():
                content = f""" Product Information:
                ProductID: {row['ProductID']}
                ProductName: {row['ProductName']}
                Description: {row['Description']}
                Price: {row['Price']}
                Stock: {row['Stock']}
                """
                metadata = {
                        "source": filepath,
                        "row_index": idx,
                        "ProductName": row["ProductName"],
                        "category": row.get("Category", "N/A"),
                        "rating": row.get("Rating", "N/A"),
                        "data_type": "product_info"
                }
                documents.append(Document(page_content=content, metadata=metadata))
        return documents
```
This method enables rich, customized document creation for downstream tasks.

---

## Summary Table

| Strategy      | Automation | Customization | Metadata Richness | Use Case                        |
|---------------|------------|---------------|-------------------|----------------------------------|
| CSVLoader     | High       | Low           | Basic             | Simple row-based document loading|
| Custom Loader | Medium     | High          | Advanced          | Complex or tailored document needs|

Choose the strategy that best fits your data complexity and processing requirements.

In [ ]:
# Create sample data for Products.csv provide some realtime product data
import pandas as pd

data = {
    "ProductID": [1, 2, 3],
    "ProductName": ["Laptop", "Smartphone", "Tablet"],
    "Price": [999.99, 499.99, 299.99],
    "Stock": [50, 200, 150],
    "Rating": [4.5, 4.7, 4.3],
    "Description": [
        "A high-performance laptop suitable for all your computing needs.",
        "A sleek and powerful smartphone with cutting-edge features.",
        "A lightweight tablet perfect for browsing and media consumption."
    ]
} 

df = pd.DataFrame(data)
df.to_csv("data/csv_files/Products.csv", index=False)

In [14]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

csv_loader = CSVLoader(file_path="data/csv_files/Products.csv", 
encoding="utf-8",
csv_args={
    'delimiter': ',',
    'quotechar': '"'
}
)
csv_documents = csv_loader.load()
print(f"Loaded {len(csv_documents)} documents using CSVLoader.")
# print("Sample document content:")
# print(csv_documents[0])  # Print the first document's content 
print(csv_documents[0].page_content)  # Print the second document's content
# Metadata Example
print("\nSample document metadata:")
for key, value in csv_documents[0].metadata.items():
    print(f"{key}: {value}")

Loaded 3 documents using CSVLoader.
ProductID: 1
ProductName: Laptop
Price: 999.99
Stock: 50
Rating: 4.5
Description: A high-performance laptop suitable for all your computing needs.

Sample document metadata:
source: data/csv_files/Products.csv
row: 0


In [22]:
# Custom csv processing for better handling
from typing import List
from langchain_core.documents import Document

def custom_csv_loader(filepath: str) -> List[Document]:
    df = pd.read_csv(filepath)
    documents = []
    for idx, row in df.iterrows():
        content = f""" Product Information:
        ProductID: {row['ProductID']}
        ProductName: {row['ProductName']}
        Description: {row['Description']}
        Price: {row['Price']}
        Stock: {row['Stock']}
        """
        metadata = {
            "source": filepath,
            "row_index": idx,
            "ProductName": row["ProductName"],
            "category": row.get("Category", "N/A"),
            "rating": row.get("Rating", "N/A"),
            "data_type": "product_info"
        }
        documents.append(Document(page_content=content, metadata=metadata))
    return documents

In [24]:
custom_csv_ldr = custom_csv_loader("data/csv_files/Products.csv")
print(f"Loaded {len(custom_csv_ldr)} documents using custom_csv_loader.")
custom_csv_ldr

Loaded 3 documents using custom_csv_loader.


[Document(metadata={'source': 'data/csv_files/Products.csv', 'row_index': 0, 'ProductName': 'Laptop', 'category': 'N/A', 'rating': 4.5, 'data_type': 'product_info'}, page_content=' Product Information:\n        ProductID: 1\n        ProductName: Laptop\n        Description: A high-performance laptop suitable for all your computing needs.\n        Price: 999.99\n        Stock: 50\n        '),
 Document(metadata={'source': 'data/csv_files/Products.csv', 'row_index': 1, 'ProductName': 'Smartphone', 'category': 'N/A', 'rating': 4.7, 'data_type': 'product_info'}, page_content=' Product Information:\n        ProductID: 2\n        ProductName: Smartphone\n        Description: A sleek and powerful smartphone with cutting-edge features.\n        Price: 499.99\n        Stock: 200\n        '),
 Document(metadata={'source': 'data/csv_files/Products.csv', 'row_index': 2, 'ProductName': 'Tablet', 'category': 'N/A', 'rating': 4.3, 'data_type': 'product_info'}, page_content=' Product Information:\n   

# Excel Processing Techniques in LangChain

LangChain supports flexible strategies for loading and processing Excel files, enabling workflows that leverage multiple sheets, rich metadata, and custom document creation.

---

## 1. UnstructuredExcelLoader

- **Approach:**  
    Loads entire Excel files, including multiple sheets, as unstructured documents.
- **Implementation:**  
    Uses `UnstructuredExcelLoader` to read `.xlsx` files and create document objects for each sheet.
- **Advantages:**  
    - Handles multiple sheets automatically.
    - Preserves sheet names and structure in metadata.
    - Useful for exploratory analysis and multi-sheet reporting.
- **Limitations:**  
    - Less control over individual cell or row formatting.
    - May require post-processing for granular data extraction.

**Example:**
```python
from langchain.document_loaders.excel import UnstructuredExcelLoader

loader = UnstructuredExcelLoader("data/excel_files/Products.xlsx")
documents = loader.load()
```

---

## 2. Custom Excel Loader

- **Approach:**  
    Use pandas to read Excel files, process sheets and rows, and create tailored document objects.
- **Implementation:**  
    Read each sheet with pandas, iterate over rows, and construct `Document` objects with custom content and metadata.
- **Advantages:**  
    - Full control over which sheets and rows to process.
    - Can combine data from multiple sheets.
    - Enables advanced filtering, summaries, and metadata enrichment.
- **Limitations:**  
    - Requires more code and maintenance.
    - Slightly slower for large files.

**Example:**
```python
import pandas as pd
from langchain_core.documents import Document

excel_file = "data/excel_files/Products.xlsx"
xls = pd.ExcelFile(excel_file)
documents = []
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    for idx, row in df.iterrows():
        content = f"Sheet: {sheet_name}\nProduct: {row.get('ProductName', '')}\nPrice: {row.get('Price', '')}"
        metadata = {"sheet": sheet_name, "row_index": idx, "source": excel_file}
        documents.append(Document(page_content=content, metadata=metadata))
```

---

## Summary Table

| Technique                | Multi-Sheet Support | Customization | Metadata Richness | Use Case                  |
|--------------------------|--------------------|---------------|-------------------|---------------------------|
| UnstructuredExcelLoader  | ✅                 | Low           | Basic             | Quick multi-sheet loading |
| Custom Loader (pandas)   | ✅                 | High          | Advanced          | Tailored document creation|

Choose the technique that best fits your Excel data complexity and processing needs.

In [25]:
# Save as Excel with multiple sheets
# Create sample data for Products.csv provide some realtime product data
import pandas as pd

data = {
    "ProductID": [1, 2, 3],
    "ProductName": ["Laptop", "Smartphone", "Tablet"],
    "Price": [999.99, 499.99, 299.99],
    "Stock": [50, 200, 150],
    "Rating": [4.5, 4.7, 4.3],
    "Description": [
        "A high-performance laptop suitable for all your computing needs.",
        "A sleek and powerful smartphone with cutting-edge features.",
        "A lightweight tablet perfect for browsing and media consumption."
    ]
} 

df = pd.DataFrame(data) 
with pd.ExcelWriter("data/excel_files/Products.xlsx") as writer:
    df.to_excel(writer, sheet_name="Products", index=False)
    df[['ProductID', 'ProductName', 'Price']].to_excel(writer, sheet_name="Summary", index=False)
    df[['ProductID', 'Stock', 'Rating']].to_excel(writer, sheet_name="Inventory", index=False)

In [26]:
# Custom excel loader for multiple sheets
def custom_excel_loader(filepath: str) -> List[Document]:

    # Read the Excel file with all sheets
    xls = pd.ExcelFile(filepath)
    documents = []
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        # create a document for each sheet
        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Columns: {', '.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n"
        sheet_content += df.to_string(index=False)

        document = Document(
            page_content=sheet_content,
            metadata={
                "source": filepath,
                "sheet_name": sheet_name,
                "num_rows": len(df),
                "num_columns": len(df.columns),
                "data_type": "excel_sheet"
            }
        )
        documents.append(document)
    return documents

In [27]:
excel_documents = custom_excel_loader("data/excel_files/Products.xlsx")
print(f"Loaded {len(excel_documents)} documents using custom_excel_loader.")
excel_documents

Loaded 3 documents using custom_excel_loader.


[Document(metadata={'source': 'data/excel_files/Products.xlsx', 'sheet_name': 'Products', 'num_rows': 3, 'num_columns': 6, 'data_type': 'excel_sheet'}, page_content='Sheet: Products\nColumns: ProductID, ProductName, Price, Stock, Rating, Description\nRows: 3\n ProductID ProductName  Price  Stock  Rating                                                      Description\n         1      Laptop 999.99     50     4.5 A high-performance laptop suitable for all your computing needs.\n         2  Smartphone 499.99    200     4.7      A sleek and powerful smartphone with cutting-edge features.\n         3      Tablet 299.99    150     4.3 A lightweight tablet perfect for browsing and media consumption.'),
 Document(metadata={'source': 'data/excel_files/Products.xlsx', 'sheet_name': 'Summary', 'num_rows': 3, 'num_columns': 3, 'data_type': 'excel_sheet'}, page_content='Sheet: Summary\nColumns: ProductID, ProductName, Price\nRows: 3\n ProductID ProductName  Price\n         1      Laptop 999.99\n  

In [28]:
from langchain_community.document_loaders import UnstructuredExcelLoader

try:
    excel_loader = UnstructuredExcelLoader(file_path="data/excel_files/Products.xlsx", 
                                           # sheet_name="Products"
                                           mode ="elements"
                                           )
    excel_documents = excel_loader.load()
    print(f"Loaded {len(excel_documents)} documents using UnstructuredExcelLoader.")
    excel_documents
except Exception as e:
    print(f"Error loading Excel file: {e}")


Error loading Excel file: No module named 'msoffcrypto'
